In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [6]:
print("torch version: {} \ntorchvision version: {}".format(torch.__version__, torchvision.__version__))

torch version: 1.0.1 
torchvision version: 0.2.1


In [7]:
model_names = ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception', 'shufflenet']

In [8]:
tran_filenames = os.listdir('data/train')
datasetdir = os.path.join('./data')
traindir = os.path.join(datasetdir,'train2')
testdir = os.path.join(datasetdir,'test2')

model_name = "densenet"
num_classes = 2
batch_size = 32
epochs = 10
feature_extract = True
trick = False
csv_name = "submission-{model_name}-e{epochs}b{batch_size}-t{trick}.csv"

In [5]:
def train(model, dataloaders, criterion, optimizer, epochs=25, is_inception=False):
    start = time.time()
    val_acc = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch, epochs-1))
        print("-"*20)
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_correts = 0.0
            for x, y in dataloaders[phase]:
                x = x.to(device)    # divice
                y = y.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == "train":
                        y_, y_aux = model(x)
                        loss1 = criterion(y_, y)
                        loss2 = criterion(y_aux, y)
                        loss = loss1 + loss2
                    else:
                        y_ = model(x)
                        loss = criterion(y_, y)
                    _, preds = torch.max(y_, 1)
                running_loss += loss.items() * x.size(0)
                running_correts += torch.sum(preds==y.data)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_correts / len(dataloaders[phase].dataset)
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:   # best_acc
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc.append(epoch_acc)
        print()
    cost = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(cost // 60, cost % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc
                    

In [6]:
def set_parameter_requires_grad(model, is_feature_extract): # 特征提取
    if is_feature_extract:
        for para in model.parameters():
            para.requires_grad = Fales